In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import metrics

In [2]:
directory = '/Users/filippo/Developer/tesi/results/hSBM/1000doc_0.5occ_fpkmweight_thr/topsbm'

In [3]:
l = 3

In [4]:
df_topics = pd.read_csv("%s/topsbm_level_%d_topics.csv"%(directory,l), index_col=None)
df_topics.head()

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,...,Topic 164,Topic 165,Topic 166,Topic 167,Topic 168,Topic 169,Topic 170,Topic 171,Topic 172,Topic 173
0,ENSG00000142224,ENSG00000135903,ENSG00000036828,ENSG00000227234,ENSG00000172995,ENSG00000170044,ENSG00000170054,ENSG00000229415,ENSG00000009709,ENSG00000215612,...,ENSG00000269586,ENSG00000182816,ENSG00000231924,ENSG00000243137,ENSG00000184351,ENSG00000186526,ENSG00000198049,ENSG00000198681,ENSG00000215148,ENSG00000237412
1,ENSG00000217236,ENSG00000004846,ENSG00000004939,ENSG00000212724,ENSG00000022355,ENSG00000162763,ENSG00000111732,ENSG00000257520,NaN,ENSG00000134595,...,ENSG00000181433,ENSG00000196156,ENSG00000183269,ENSG00000204941,NaN,ENSG00000186723,ENSG00000248713,NaN,NaN,NaN
2,ENSG00000004809,ENSG00000077498,ENSG00000074803,ENSG00000198573,ENSG00000215397,ENSG00000165694,ENSG00000185933,ENSG00000008197,NaN,ENSG00000011083,...,ENSG00000268940,NaN,ENSG00000188784,ENSG00000183668,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000161973,ENSG00000105679,ENSG00000151418,ENSG00000006059,ENSG00000101327,ENSG00000006659,ENSG00000196604,NaN,NaN,ENSG00000183090,...,ENSG00000228836,NaN,ENSG00000188076,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000203784,ENSG00000134160,ENSG00000162399,ENSG00000131738,ENSG00000006116,ENSG00000119913,ENSG00000124196,NaN,NaN,ENSG00000269466,...,NaN,NaN,ENSG00000197416,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cluster={}
for i,c in enumerate(df_topics.columns):
    cluster[i]=df_topics[c].dropna().values